# Import the necessary package


In [ ]:
#Install the country-converter

!pip install country-converter


In [ ]:
import pandas as pd
import plotly.express as px
from country_converter import CountryConverter



In [ ]:
df = pd.read_csv("mydata/admissions_dataset.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

# Aggregate Applicant by country

In [ ]:
df_nationality = df["CountryISO2"].value_counts(ascending=False)
df_nationality_df = df_nationality.reset_index()
df_nationality_df.columns = ['CountryISO2', 'Count']

print(df_nationality_df)

# data enrichment using the countryconverter library

In [ ]:
cc = CountryConverter()
df_nationality_df["country_name"] = cc.convert(df_nationality_df["CountryISO2"], to="name_short")
df_nationality_df

# Applicant Nationality Barchart

In [ ]:
# create a horizontal barchart
fig = px.bar(data_frame= df_nationality_df,
             x = "Count",
             y="country_name",
             orientation="h",
             title = "Applicants: Nationality"
)
# Set axis labels
fig.update_layout(xaxis_title="Frequency [count]", yaxis_title="Country",
                   yaxis=dict(
        categoryorder='total ascending'  # 'ascending' puts the largest value at the top for horizontal bars
    )
)
# Show the plot
fig.show()

# Normalize Nationality

In [ ]:
df_nationality_df["count_pct"] = (
    (df_nationality_df["Count"]/df_nationality_df["Count"].sum()) * 100
)
df_nationality_df

# Normalize Barchart

In [ ]:
# create a horizontal barchart
fig = px.bar(data_frame= df_nationality_df,
             x = "count_pct",
             y="country_name",
             orientation="h",
             title = "Applicants: Nationality"
)
# Set axis labels
fig.update_layout(xaxis_title="Frequency [count]", yaxis_title="Country",
                   yaxis=dict(
        categoryorder='total ascending'  # 'ascending' puts the largest value at the top for horizontal bars
    )
)
# Show the plot
fig.show()

In [ ]:
df_nationality_df["CountryISO3"] = cc.convert(df_nationality_df["CountryISO2"], to="ISO3")
df_nationality_df

# Create a function build_nat_choropleth that returns plotly choropleth map showing the "count" of DS applicants in each country in the globe.

In [ ]:
def build_nat_choropleth():
  fig = px.choropleth(
      data_frame=df_nationality_df,
      locations="CountryISO3",
      locationmode="ISO-3",
      color="count_pct",
      projection="natural earth",
      color_continuous_scale=px.colors.sequential.Oranges,
      title="Applicants: Nationality",
      hover_name="country_name",
      hover_data=["Count", "count_pct"],
  )
  return fig
nat_fig = build_nat_choropleth()
nat_fig.show()

#  Age of Applicants

In [ ]:
import pandas as pd
from datetime import datetime


# Convert 'birthday' to datetime
df['birthday'] = pd.to_datetime(df['birthday'], dayfirst=True)


In [ ]:
today = pd.to_datetime("today")
df['age'] = df['birthday'].apply(
    lambda dob: today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    )


In [ ]:
df["age"].head()

# Applicant Age histogram

In [ ]:
def build_age_hist():
  fig = px.histogram(x = df["age"], nbins = 20, title = "Applicants: Age")
  fig.update_layout(xaxis_title="Age", yaxis_title="Frequency [count]",width=800,
        height=600  )
  return fig
age_fig = build_age_hist()
age_fig.show()

In [ ]:
import plotly.express as px

def build_age_hist():
    fig = px.histogram(x = df["age"], title = "Applicants: Age")

    # Define custom bin sizes and range
    fig.update_traces(
        xbins=dict(
            start=0,
            end=100,
            size=10
        )
    )

    fig.update_layout(
        xaxis_title="Age",
        yaxis_title="Frequency [count]",
        width=800,
        height=600
    )

    return fig

age_fig = build_age_hist()
age_fig.show()


# Education

In [ ]:
education = df["highest_degree_earned"].value_counts(ascending = False)
education

# Reading the result into DataFrame

In [ ]:
# Convert the Series to a DataFrame
education_df = education.reset_index()

# Rename columns for clarity
education_df.columns = ['Degree', 'Count']

# Display the resulting DataFrame
print(education_df)

In [ ]:
def ed_sort(counts):
    """Sort array `counts` from highest to lowest degree earned."""
    # Create a mapping for the degrees to numeric values
    # PhD -> 3 (highest), Masters -> 2, Bachelors -> 1 (lowest)
    mapping = {
        "Bachelors": 1,
        "Masters": 2,
        "PhD": 3,
    }

    # Sort the degrees based on the mapping and return the corresponding sort order
    sort_order = counts.index.map(mapping)

    return sort_order

# Assuming 'education' is a Series with degrees as the index (degree names)
education = df["highest_degree_earned"].value_counts(ascending=False)

# Create the mapping dictionary again for sorting purposes
mapping = {
    "Bachelors": 1,
    "Masters": 2,
    "PhD": 3,
}

# Sort the education Series based on the custom sort order
education_sorted = education.iloc[education.index.map(lambda x: mapping.get(x, 0)).argsort()]

# Display the sorted education counts
print(education_sorted)


In [ ]:
import plotly.express as px

def build_ed_bar():
    # Create bar chart
    fig = px.bar(
        education_sorted,  # Data for the bar chart
        x=education_sorted.index,  # X-axis (degree names)
        y=education_sorted.values,  # Y-axis (counts)
        title="Applicants : Highest Degree Earned",  # Title of the chart
        labels={"x": "Degree", "y": "Number of Applicants"}  # Axis labels
    )

    # Optionally: Reverse the x-axis if you want to display it in descending order
    fig.update_layout(
        xaxis_title="Degree",
        yaxis_title="Frequency [count]",
        xaxis_categoryorder='total descending'  # Sort from highest to lowest degree
    )

    return fig

# Generate the figure and show it
ed_fig = build_ed_bar()
print("ed_fig type:", type(ed_fig))  # Confirm the type of the figure object
ed_fig.show()  # Display the figure


# Aggregate Applicants by quiz completion

In [ ]:
results = df["admission_quiz"].value_counts(ascending = False)
results

In [ ]:
# Get the counts safely (default to 0 if not present)
incomplete = results.get("Not Completed", 0)
completed = results.get("Completed", 0)

print(f"Number of applicants who completed the quiz: {completed}")
print(f"Number of applicants who did not complete the quiz: {incomplete}")

In [ ]:
total = incomplete + completed
prop_incomplete = incomplete / total
print(
    "Proportion of users who don't complete admissions quiz:", round(prop_incomplete, 2)
)

# Develop the Hypothesis

In [ ]:
null_hypothesis = '''
There is no relationship between sending an email and completing the admissions quiz.
sending an email to 'to no quiz applicants' does  not affect the completion of the quiz.
'''

alternate_hypothesis = """
There is a relationship between sending an email and completing the admissions quiz.
sending an email to 'to quiz applicants' does affect the completion of the quiz.
"""

print("Null Hypothesis:", null_hypothesis)
print("Alternate Hypothesis:", alternate_hypothesis)

# filter the data so that we're only looking at students who applied on a certain date.

In [ ]:
import pandas as pd

def find_by_date(df, date_string=None, start_date=None, end_date=None, mode='day'):
     # Normalize the quiz status
    df["admission_quiz"] = df["admission_quiz"].str.lower().str.strip()
    df["created_at"] = pd.to_datetime(df["created_at"])

    if mode == 'day':
        start = pd.to_datetime(date_string)
        end = start + pd.Timedelta(days=1)

    elif mode == 'month':
        date = pd.to_datetime(date_string)
        start = pd.Timestamp(date.year, date.month, 1)
        end = start + pd.offsets.MonthEnd(1)

    elif mode == 'range':
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date) + pd.Timedelta(days=1)

    else:
        raise ValueError("Invalid mode. Use 'day', 'month', or 'range'.")

    # Filter
    filtered = df[
        (df["created_at"] >= start) &
        (df["created_at"] < end) &
        (df["admission_quiz"] == "not completed")
    ]

    return filtered


In [ ]:
# For a specific day
find_by_date(df, date_string="2023-08-15", mode='day')

# For a month
find_by_date(df, date_string="2023-08", mode='month')

# For a range
find_by_date(df, start_date="2023-08-01", end_date="2023-08-20", mode='range')


In [ ]:
result = find_by_date(df, date_string="2023-08-10", mode='day')
print(result)


In [ ]:
find_by_date(df, date_string="2023-08", mode='month')

# For getting a list

In [ ]:
observations_ = find_by_date(df, date_string="2023-11-29", mode='day')

print("observations type:", type(observations_))
print("observations len:", len(observations_))

# Check if observations is empty before accessing the first element
if len(observations_) > 0:
    print(observations_.iloc[0])  # Access the first row safely
else:
    print("No observations found.")


#Transform:  Designing the Experiment

we're going to be figuring out which students didn't take the quiz, and assigning them to different experimental groups

In [ ]:

def get_incomplete_quiz_applicants(df):
    """
    Returns a list of applicants who did not complete the quiz.

    Parameters
    ----------
    df : pandas.DataFrame
        The input DataFrame with at least 'admission_quiz' column.

    Returns
    -------
    List[dict]
        List of dictionaries containing the rows of applicants with 'Not Completed' quiz status.
    """
    # Normalize quiz column to avoid casing/spacing issues
    #df["admission_quiz"] = df["admission_quiz"].str.strip().str.lower()

    # Filter for 'not completed'
    not_completed_df = df[df["admission_quiz"] == "not completed"]

    # Convert to list of dicts
    not_completed_list = not_completed_df.to_dict(orient="records")

    return not_completed_list


In [ ]:
observations = get_incomplete_quiz_applicants(df)

print(f"Found {len(observations)} applicants who did not complete the quiz.")
print(observations[:3])  # preview first 3


In [ ]:
import random

In [ ]:
random.seed(42)
random.shuffle(observations)

In [ ]:
idx = len(observations)//2

In [ ]:
len(observations[idx:])

In [ ]:
def assign_to_groups(observations):
    """Randomly assigns observations to control and treatment groups.

    Parameters
    ----------
    observations : list or pymongo.cursor.Cursor
        List of users to assign to groups.

    Returns
    -------
    observations : list
        List of documents from `observations` with two additional keys:
        `inExperiment` and `group`.
    """
    # Shuffle `observations`
    random.seed(42)
    random.shuffle(observations)

    # Get index position of item at observations halfway point
    idx = len(observations)//2

    # Assign first half of observations to control group
    for doc in observations[:idx]:
        doc["inExperiment"] = True
        doc["group"] = "no email (control)"

    # Assign second half of observations to treatment group
    for doc in observations[idx:]:
        doc["inExperiment"] = True
        doc["group"] = "email (treatment)"

    return observations


observations_assigned = assign_to_groups(observations)

print("observations_assigned type:", type(observations_assigned))
print("observations_assigned len:", len(observations_assigned))
observations_assigned[0]

In [ ]:
observations_assigned[-1]

# creating a class

In [ ]:
import pandas as pd

class CSVRepository:
    """Repository class for interacting with applicant data from a CSV file.

    Parameters
    ----------
    filepath : str
        Path to the CSV file containing applicant data.

    Attributes
    ----------
    df : pandas.DataFrame
        All data will be extracted from and loaded to this DataFrame.
    """

    def __init__(self, filepath="mydata/admissions_dataset.csv"):
        self.df = pd.read_csv(filepath)
        self.df["created_at"] = pd.to_datetime(self.df["created_at"])
        self.df["admission_quiz"] = self.df["admission_quiz"].str.strip().str.lower()

    # Normalize column values
    def find_by_date(self, date_string=None, start_date=None, end_date=None, mode='day'):
        df = self.df

        if mode == 'day':
            start = pd.to_datetime(date_string)
            end = start + pd.Timedelta(days=1)

        elif mode == 'month':
            date = pd.to_datetime(date_string)
            start = pd.Timestamp(date.year, date.month, 1)
            end = start + pd.offsets.MonthEnd(1)

        elif mode == 'range':
            start = pd.to_datetime(start_date)
            end = pd.to_datetime(end_date) + pd.Timedelta(days=1)
        else:
            raise ValueError("Invalid mode. Use 'day', 'month', or 'range'.")

        # Filter
        filtered = df[
            (df["created_at"] >= start) &
            (df["created_at"] < end) &
            (df["admission_quiz"] == "not completed")
        ]

        return filtered
    # # Task 7.2.18
    # def find_not_completed(self, start=None, end=None):
    #     """Returns applicants who did not complete the quiz.

    #     Parameters
    #     ----------
    #     start : str, optional
    #         Start date in 'YYYY-MM-DD' format.
    #     end : str, optional
    #         End date in 'YYYY-MM-DD' format.

    #     Returns
    #     -------
    #     list
    #         List of applicant records as dictionaries.
    #     """
    #     df = self.df

    #     if start and end:
    #         start_date = pd.to_datetime(start)
    #         end_date = pd.to_datetime(end)
    #         df = df[(df["created_at"] >= start_date) & (df["created_at"] <= end_date)]

    #     not_completed_df = df[df["admission_quiz"] == "not completed"]
    #     return not_completed_df.to_dict(orient="records")

    # # Task 7.2.19
    # def count_not_completed(self):
    #     """Counts how many applicants did not complete the quiz.

    #     Returns
    #     -------
    #     int
    #         Count of records where quiz is not completed.
    #     """
    #     return len(self.df[self.df["admission_quiz"] == "not completed"])


# Instantiate the Repository

In [ ]:
repo = CSVRepository()
print("repo type:", type(repo))
repo

In [ ]:
dir(repo)

In [ ]:
c_test = repo.df
print("c_test type:", type(c_test))
#c_test

# find_by_date method

Our class is built, and now we need to take the ETL functions we created and turn them into class methods

In [ ]:
repo = CSVRepository()

# Get applicants on a specific day
day_result = repo.find_by_date(date_string="2023-11-29", mode="day")
print("Results for 2023-11-29:", len(day_result))

# Get for a month
month_result = repo.find_by_date(date_string="2023-11", mode="month")
print("Results for November:", len(month_result))

# Get for a range
range_result = repo.find_by_date(start_date="2023-11-01", end_date="2023-11-30", mode="range")
print("Results for Nov range:", len(range_result))


In [ ]:
nov_11_users = repo.find_by_date(date_string="2023-11-29", mode="day")
print("nov_11_users type", type(nov_11_users))
print("nov_11_users len", len(nov_11_users))
nov_11_users[:3]

# Statistical Power

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import scipy
from statsmodels.stats.contingency_tables import Table2x2
from statsmodels.stats.power import GofChisquarePower
#from teaching_tools.ab_test.experiment import Experiment

# Power Calculation
Instantiate a GofChisquarePower object and assign it to the variable name chi_square_power. Then use it to calculate the group_size needed to detect an effect size of 0.2, with an alpha of 0.05 and power of 0.8

In [ ]:
chi_square_power = GofChisquarePower()
group_size = int(
    chi_square_power.solve_power(effect_size=0.2, alpha=0.05, power=0.8))

print("Group size:", group_size)
print("Total # of applicants needed:", group_size * 2)

#  Power Curve Plot

In [ ]:
n_observations = np.arange(0, group_size * 2 + 1)
effect_sizes = np.array([0.2, 0.5, 0.8])

#plot power curve uisng chi_square_power

chi_square_power.plot_power(
    dep_var="nobs",
    nobs=n_observations,
    effect_size=effect_sizes,
    alpha=0.05,
    n_bins = 2
)

# Aggregate Applicant per day
we decided that our experiment would focus on the subset of applicants who don't take the admissions quiz immediately after creating an account. We know we need around 400 observations from this subset, but how long do we need to run our experiment for in order to get that number?

To answer that question, we first need to calculate how many such applicants open an account each day

In [ ]:

# Convert list of dicts to DataFrame
observations_df = pd.DataFrame(observations)

# Make sure created_at is datetime
observations_df["created_at"] = pd.to_datetime(observations_df["created_at"])

# Group by day and count number of records
daily_counts = (
    observations_df
    .groupby(observations_df["created_at"].dt.date)
    .size()
    .reset_index(name="count")
)

# Display the result
print("result type:", type(daily_counts))
print(daily_counts.head())


In [ ]:
no_quiz = (daily_counts.rename({
    "created_at":"date", "count":"users"}, axis = 1)
    .set_index("date")
    .sort_index()
    .squeeze()
)

print("no_quiz type:", type(no_quiz))
print("no_quiz shape:", no_quiz.shape)
no_quiz.head()

# Distribution of Applicants per day

In [ ]:
no_quiz.hist()
plt.xlabel("No-quiz Applicants")
plt.ylabel("Frequency [count]")
plt.title("Distribution of Daily No-quiz Applicants")

# Average number of applicants per day who did not complete the quiz.

Mean (1.39):
On average, about 1.39 users per day did not complete the quiz.

Standard Deviation (std) (0.62):
On any given day, the number of incomplete applicants typically varies by about 0.62 users from the average. This indicates relatively low day-to-day fluctuation.

In [ ]:
no_quiz.mean()

In [ ]:
mean = no_quiz.describe()["mean"]
std = no_quiz.describe()["std"]
print("no_quiz mean:", mean)
print("no_quiz std:", std)

# how many days do we need to run the experiment to make sure we get to 400 users?

Probability Distribution

In [ ]:
import numpy as np

# Given
mean_daily = 1.38544474393531
std_daily = 0.6235933394605798
target_total = 400

# Estimate number of days needed
estimated_days = int(np.ceil(target_total / mean_daily))

# Compute mean and std of the sum for a given number of days
days = estimated_days
sum_mean = days * mean_daily
sum_std = np.sqrt(days) * std_daily

print("Estimated days needed:", days)
print("Mean of sum:", sum_mean)
print("Std of sum:", sum_std)


# Cumulative Density Function
Calculate the probability of getting 400 or more sign-ups over 289 days

In [ ]:
prob_400_or_fewer = scipy.stats.norm.cdf(
    group_size * 2,
    loc = sum_mean,
    scale = sum_std
)
prob_400_or_greater = 1 - prob_400_or_fewer

print(
    f"Probability of getting 400+ no_quiz in {days} days:",
    round(prob_400_or_greater, 3),
)


# Running the Experiment

In [ ]:
# from teaching_tools.ab_test.experiment import Experiment

In [ ]:
# Convert list of dicts to DataFrame
observations_df = pd.DataFrame(observations)

# Make sure created_at is datetime
observations_df["created_at"] = pd.to_datetime(observations_df["created_at"])

def run_experiment(observations, days=100, seed=42):
    np.random.seed(seed)

    # Group by day
    daily_counts = (
        observations.groupby(observations["created_at"].dt.date)
        .size()
        .reset_index(name="users")
        .rename(columns={"created_at": "date"})
        .sort_values("date")
    )

    # Ensure we have enough days
    if days > len(daily_counts):
        raise ValueError(f"Not enough days in the dataset. You asked for {days}, but only have {len(daily_counts)}.")

    # Sample days randomly
    sampled_days = daily_counts.sample(n=days, replace=False, random_state=seed)

    total_users = sampled_days["users"].sum()

    return {
        "days_sampled": days,
        "total_users": total_users,
        "daily_sample": sampled_days
    }

# Now run
result = run_experiment(observations_df, days=371)
print("result type:", type(result))
print(f"Total users in {result['days_sampled']} days:", result['total_users'])
result["daily_sample"].head()


# Evaluating Experiment Results
Build Contingency Table

In [ ]:
observations_df.tail()

In [ ]:
observations_df[["group","admission_quiz"]]

In [ ]:
observations_df["group"].value_counts()


# Simulate conversion outcome

In [ ]:
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Check what your actual group labels are
print("Group labels:", observations_df["group"].unique())

# Assign conversion probabilities matching those exact labels
conversion_rate = {
    "no email (control)": 0.15,    # 15% completion in control
    "email (treatment)":   0.25     # 25% completion in treatment
}

# Simulate a “converted” flag
def simulate_conversion(group_label):
    """Return True if user converts based on their group's probability."""
    return np.random.rand() < conversion_rate[group_label]

observations_df["converted"] = observations_df["group"].apply(simulate_conversion)

# Quick sanity check
contingency = pd.crosstab(observations_df["group"], observations_df["converted"])
contingency.columns = ["Not Completed", "Completed"]
print("\nContingency Table:")
print(contingency)


In [ ]:


def build_contingency_bar(contingency):
    """
    Create a side-by-side bar chart showing completed vs not completed
    for control and treatment groups.

    Parameters
    ----------
    contingency : pandas.DataFrame
        A 2×2 contingency table with index = group and columns = ['Not Completed', 'Completed'].

    Returns
    -------
    fig : plotly.graph_objs._figure.Figure
    """
    # Convert to long form for easier plotting
    df_plot = (
        contingency
        .reset_index()
        .melt(id_vars="group", value_vars=["Not Completed", "Completed"],
              var_name="Outcome", value_name="Count")
    )

    fig = px.bar(
        df_plot,
        x="group",
        y="Count",
        color="Outcome",
        barmode="group",
        title="Quiz Completion by Group"
    )
    fig.update_layout(
        xaxis_title="Group",
        yaxis_title="Number of Users",
        legend_title="Outcome"
    )
    return fig

# Usage
fig = build_contingency_bar(contingency)
fig.show()


# In order to determine if this difference is more than random variation, we need to take our results, put them into a contingency table, and run a statistical test.


## Instantiate a Table2x2 object named contingency_table, using the values from the data you created in the previous task.

In [ ]:
# Extract into a 2×2 numpy array:
counts = np.array([
    [
        contingency.loc["no email (control)", "Not Completed"],
        contingency.loc["no email (control)", "Completed"]
    ],
    [
        contingency.loc["email (treatment)", "Not Completed"],
        contingency.loc["email (treatment)", "Completed"]
    ]
], dtype=int)

# 2. Instantiate Table2x2
contingency_table = Table2x2(counts)

# 3. Inspect
print("contingency_table type:", type(contingency_table))
print("Original 2×2 array (table_orig):\n", contingency_table.table_orig)

# Calculate the fitted values for your contigency_table

In [ ]:
contingency_table.fittedvalues

# Calculate the joint probabilities under independence for your contingency_table

In [ ]:
# # Calculate independent joint probabilities
contingency_table.independence_probabilities.round(3)

# Conduct Chi-Square Test

All the previous calculations have shown us that some of the people who got an email went on to complete the quiz, but we don't know what might be driving that effect. After all, some people might be responding to getting an email, but others might have finished the quiz whether we emailed them or not. Either way, the effect we found could just as easily be due to chance as it could be a result of something we did. The only way to find out whether the result is due to chance is to calculate statistical significance.


In [ ]:
chi_square_test = contingency_table.test_nominal_association()

print("chi_square_test type:", type(chi_square_test))
print(chi_square_test)

The important part of that result is the p-value. We set our threshold for significance at 0.05 way back at the beginning, so, for our results to be statistically significant, the p-value needs to be less than or equal to 0.05.

Our p-value is 0.0258, which is less than 0.05. This means that the difference we saw in our side-by-side bar graph is unlikely to be due to chance. In other words, it appears to be a real effect — a signal, not just noise.

So, we reject the null hypothesis and conclude that the email treatment had a statistically significant effect on conversion.

# Calculating Practical Impact

Setting the issue of statistical significance aside for now, there's one more calculation that helps in interpreting and communicating the impact: the odds ratio. This tells us how much more likely someone in the treatment group is to complete the quiz compared to someone in the control group.

In [ ]:
odds_ratio = contingency_table.oddsratio.round(1)
print("Odds ratio:", odds_ratio)

Odds Ratio Interpretation:

The odds ratio in our experiment is 1.7. This means that for every 1 person in the control group who completes the quiz, approximately 1.7 people in the treatment group do. In simpler terms, someone in the email treatment group is 70% more likely to complete the quiz compared to someone in the no email control group.

Unlike in cases where the result is not statistically significant, here our p-value was 0.0258, which is below the 0.05 threshold. That means the difference we observed is unlikely to be due to random chance. So, in this case, the odds ratio is both statistically and practically meaningful, and it provides a useful summary of the treatment’s effect.

However, as always, it's important to consider context:

Is the 70% increase in odds large enough to justify a real-world change in behavior (like running a daily email campaign)?

Are there costs or tradeoffs involved in implementing the treatment that outweigh the benefits?

This is where practical significance must complement statistical significance when making business decisions.

# Summary

In [ ]:
summary = contingency_table.summary()
print("summary type:", type(summary))
summary